In [1]:
#!/usr/bin/env python# coding: utf-8import boto3dynamodb = boto3.client('dynamodb')table_name = 'Users'key_schema = [    {        'AttributeName': 'username',        'KeyType': 'HASH'    },    {        'AttributeName': 'last_name',        'KeyType': 'RANGE'    }]attribute_definitions = [    {        'AttributeName': 'username',        'AttributeType': 'S'    },    {        'AttributeName': 'last_name',        'AttributeType': 'S'    }]provisioned_throughput = {    'ReadCapacityUnits': 10,    'WriteCapacityUnits': 5}

In [2]:
# create tableresponse = dynamodb.create_table(    TableName=table_name,    KeySchema=key_schema,    AttributeDefinitions=attribute_definitions,    ProvisionedThroughput=provisioned_throughput)try:    table = dynamodb.describe_table(TableName=table_name)    print(f"the {table_name} table already exists")except dynamodb.exceptions.ResourceNotFoundException:    response = dynamodb.create_table(        TableName=table_name,        KeySchema=key_schema,        AttributeDefinitions=attribute_definitions,        ProvisionedThroughput=provisioned_throughput    )    print(f"{table_name} created")except Exception as e:    print(f"error: {str(e)}")

In [3]:
# get all itemsresponse = dynamodb.scan(    TableName=table_name)if 'Items' in response:    items = response['Items']    if len(items) == 0:        print(f"{table_name} is empty")    for item in items:        print(f"Item: {item}")else:    print(f"Not found items in {table_name}")

In [4]:
# get one itemusername = 'juanp06'last_name = 'perez'response = dynamodb.get_item(    TableName=table_name,    Key={        'username': {'S': username},        'last_name': {'S': last_name}    })if 'Item' in response:    item = response['Item']    print(f"item finded: {item}")else:    print(f"No se encontró un elemento con username: {username}")

In [5]:
# put itemdynamodb = boto3.client('dynamodb')table_name = 'Users'item = {    'username': 'juanp06',    'last_name': 'perez',    'first_name': 'juan',    'age': 26,    'account_type': 'user'}response = dynamodb.put_item(    TableName=table_name,    Item={        'username': {'S': item['username']},        'last_name': {'S': item['last_name']},        'first_name': {'S': item['first_name']},        'age': {'N': str(item['age'])},        'account_type': {'S': item['account_type']}    })print(f"added item")

In [6]:
# update itemusername = 'juanp06'last_name = 'perez'new_values = {    'age': 87,    'account_type': 'admin'}update_expression = "SET age = :age, account_type = :account_type"expression_attribute_values = {    ':age': {'N': str(new_values['age'])},    ':account_type': {'S': new_values['account_type']}}response = dynamodb.update_item(    TableName=table_name,    Key={        'username': {'S': username},        'last_name': {'S': last_name}    },    UpdateExpression=update_expression,    ExpressionAttributeValues=expression_attribute_values)print(f"item updated")

In [7]:
# delete itemresponse = dynamodb.delete_item(    TableName=table_name,    Key={        'username': {'S': username},        'last_name': {'S': last_name}    })print(f"item deleted")

In [8]:
# delete tabletry:    dynamodb.delete_table(TableName=table_name)    print(f"{table_name} table deleted")except Exception as e:    print(f"error: {str(e)}")

In [9]:
# scan with filtersfilter_expression = "age >= :min_age"expression_attribute_values = {    ':min_age': {'N': '40'}}response = dynamodb.scan(    TableName=table_name,    FilterExpression=filter_expression,    ExpressionAttributeValues=expression_attribute_values)if 'Items' in response:    items = response['Items']    if len(items) == 0:        print(f"not found")    for item in items:        print(f"Item: {item}")else:    print(f"Not found items in {table_name}")